In [60]:
from utils import Settings
from openai import OpenAI
import json
import requests
from langchain_openai import ChatOpenAI

api_key = Settings().openai_api_key

In [61]:
model = ChatOpenAI(model='gpt-4o-mini', openai_api_key=api_key)

In [62]:
model.invoke('testowy prompt')

AIMessage(content='Wygląda na to, że chcesz przetestować coś. Jak mogę Ci pomóc?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 10, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZDO1BeKWCXVbIBaG6jBzd533D3oH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--4813dc8c-c51a-4ad6-b20f-d5f5dd1b7647-0', usage_metadata={'input_tokens': 10, 'output_tokens': 21, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### prompt template dynamiczne wstawianie wartości do promptu

In [63]:
from langchain_core.prompts import ChatPromptTemplate

In [64]:
prompt = ChatPromptTemplate.from_template("Napisz definicję podanego słowa w sposób zrozumiały dla przedszkolaka: {word}")

In [65]:
prepared_prompt = prompt.invoke({"word": "łyżka"})
model.invoke(prepared_prompt)

AIMessage(content='Łyżka to małe narzędzie, które pomaga nam jeść zupy, jogurty i inne pyszne jedzenie. Ma długą rączkę i zaokrągloną część na końcu, która jest jak mała miseczka. Używamy jej, żeby nabrać jedzenie i włożyć do ust. ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 35, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZDO1nhsACdnhXZXEFEQU87bVRqD9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--152d35fb-cd11-47f5-9bc2-7a1d87bcd3f3-0', usage_metadata={'input_tokens': 35, 'output_tokens': 80, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_to

#### tworzenie łancucha

In [66]:
chain = prompt | model

In [ ]:
# jeśli prompt zawiera tylko jedna zmienna to mozna podac bezpośrednio string
chain.invoke('koń') 

#### batch - wiele zapytan w jednym

In [33]:
chain.batch(
    [{"word": 'łyżka'}, {'word': 'ławka'}],

    config = {
        'max_concurrency':10
    }
    
)

[AIMessage(content='Łyżka to małe narzędzie, które wygląda jak mała łyżka z uchwytami. Używamy jej, żeby nabierać jedzenie, na przykład zupy lub jogurtu. Można nią też mieszać różne potrawy. Łyżka pomaga nam jeść!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 35, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZD08T9VuI0CqL0vF1mdArDp2TVQg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e2275cd1-de9f-4c79-93c1-cb49e511fa69-0', usage_metadata={'input_tokens': 35, 'output_tokens': 70, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

#### stream - przetwarzanie strumieniowe

In [34]:
# czym jest generator 

my_list = [num for num in range(1000)] # lista liczba od 0 do 999 wszystkie dostepne w dowolnym momencie - wszytko w pamieci 
my_gen = (num for num in range(10))  # generator liczb  od 0 do 9 - funckjonalnośc gotowac do wyganerowania takiej pojedynczej lcizby w danym momencie - dostep do pojedynczego elementu wdanym momenncie
# next(my_gen)

In [35]:
for s in chain.stream({"word": 'skakanka'}):
    print(s.content, end='')

Skakanka to specjalna lina, na której można skakać. Ma dwa uchwyty po bokach, a my trzymamy je w dłoniach. Skacząc przez skakankę, możemy się świetnie bawić i ćwiczyć. Można skakać samemu albo z przyjaciółmi. To trochę jak zabawa w łatki, tylko z liną!

#### output parsers

In [54]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

parser = StrOutputParser()
json_parser = JsonOutputParser() ## przykład !!! 

prompt = ChatPromptTemplate.from_template("Napisz definicję podanego słowa w sposób zrozumiały dla przedszkolaka: {word}. Zwróć format json gdzie jako klucz podasz slowo a wartosc wyjasnienie")

chain = prompt | model | json_parser ## runnablessequence

In [55]:
chain.invoke({'word': 'krzesło'})

{'krzesło': 'Krzesło to mebel, na którym możemy usiąść. Ma cztery nogi i zazwyczaj ma oparcie, które pomaga nam w wygodnym siedzeniu. Używamy go, gdy chcemy zjeść, bawić się lub rysować.'}

#### system messages

In [71]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Napisz definicję podanego słowa w sposób zrozumiały dla przedszkolaka"),
    ("user", "Co to jest {word}"),
])
    
chain = prompt | model | parser

In [73]:
chain.invoke("pasta do butów")

'Pasta do butów to specjalna substancja, którą smarujemy nasze buty, żeby ładnie wyglądały i były błyszczące. To tak, jakbyśmy dawali butom "kremik", żeby były szczęśliwe i nie brudziły się tak łatwo. Dzięki pasty buty są bardziej odporne na deszcz i błoto!'

In [ ]:
class Person:

    def przywitaj_sie(self):
        print('hello')


class Auto:

    def start(self):
        print('Uruchomiono silnik')

In [ ]:
person = Person()
auto = Auto()

person.przywitaj_sie()
auto.start()
# auto.przywitaj_sie()

In [42]:
chain = prompt | model | auto

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class '__main__.Auto'>

In [43]:
def check_a():
    print("sprawdzam a")
    return True

def check_b():
    print("sprawdzam b")
    return True

if check_a() or check_b():
    print("OK")

sprawdzam a
OK


In [45]:
if check_a() | check_b():
    print("OK")

sprawdzam a
sprawdzam b
OK


In [44]:
result_a = check_a()
result_b = check_b()

if result_a or result_b:
    print("OK")

sprawdzam a
sprawdzam b
OK
